In [ ]:
import os
os.environ["DJANGO_SETTINGS_MODULE"] = "ClusterCast.settings"
import django
django.setup()
import sys
sys.path.append("/home/ajp031/StockDeepLearning/ClusterCast/ClusterCast")
from django.db.models.functions import Now
from asgiref.sync import sync_to_async
from tensorflow.keras.layers import Input, LSTM, Dropout, RepeatVector, TimeDistributed, Dense, Masking
from tensorflow.keras.models import Model
import ClusterPipeline.models.RNNModels as rnn
from tensorflow.keras.callbacks import EarlyStopping

import ClusterPipeline.models.ClusterProcessing as cp
import ClusterPipeline.models.RNNModels as rnn 

In [ ]:
@sync_to_async
def get_all_objects(features,steps = None):
    # Force the query to execute and load all results into memory
    params = list(cp.StockClusterGroupParams.objects.all())
    matching_groups = []
    for param in params:
        if steps is None:
            if param.cluster_features == features:
                group = cp.StockClusterGroup.objects.get(pk=param.pk)
                matching_groups.append(group)
                group.load_saved_group()

        else: 
            if param.cluster_features == features and param.n_steps == steps:
                group = cp.StockClusterGroup.objects.get(pk=param.pk)
                matching_groups.append(group)
                group.load_saved_group()
                print(group.group_params.tickers)
    
    return matching_groups


cluster_features = ["close", "bb_low", "bb_high"]
steps = 20

async def create_cluster_group_params(cluster_features, steps=20):

    cluster_groups = await get_all_objects(features=cluster_features,steps=steps)
    return cluster_groups

# Run the async function

matching_groups = await create_cluster_group_params(cluster_features, steps=steps)

In [ ]:
@sync_to_async
def get_all_clusters(groups):
    clusters = []
    features = []
    for group in groups:
        clusters += group.clusters

    for cluster in clusters:
        models = rnn.RNNModel.objects.filter(cluster=cluster)
        for model in models: 
            if model:
                features += (model.model_features)

    features = list(set(features))
    return clusters, features

clusters, all_features = await get_all_clusters(matching_groups)



In [ ]:
test_cluster = clusters[8]

In [ ]:
import random
features = random.sample(all_features, 50)

feature_indices = [test_cluster.X_feature_dict[feature] for feature in features] 
test_cluster.X_train_filtered = test_cluster.X_train[:,:,feature_indices]
test_cluster.X_test_filtered = test_cluster.X_test[:,:,feature_indices]


In [ ]:
from tensorflow.keras.models import Sequential
from keras.layers import Input, LSTM, Dropout, TimeDistributed, Dense, Concatenate, Permute, Reshape, Multiply
from tensorflow.keras.optimizers import Adam
import keras.backend as K
from keras.layers import Layer
from keras.layers import Activation, Flatten
# Custom Attention Layer

def create_model_with_attention2(input_shape, latent_dim=6):
    # Input layer
    input_layer = Input(shape=(None, input_shape))

    # Encoder

    encoder_lstm1 = LSTM(units=50, activation='tanh', return_sequences=True, name='encoder_lstm_1_restore')(input_layer)
    encoder_dropout1 = Dropout(0.2, name='encoder_dropout_1_restore')(encoder_lstm1)

    encoder_lstm2 = LSTM(units=25, activation='tanh', return_sequences=True, name='encoder_lstm_2_restore')(encoder_dropout1)
    encoder_dropout2 = Dropout(0.2, name='encoder_dropout_2_restore')(encoder_lstm2)

    encoder_lstm3 = LSTM(units=10, activation='tanh', return_sequences=True, name='encoder_lstm_3_restore')(encoder_dropout2)
    encoder_dropout3 = Dropout(0.2, name='encoder_dropout_3_restore')(encoder_lstm3)

    output_lstm = LSTM(units=6, activation='tanh', return_sequences=True, name='outputLSTM')(encoder_dropout3)
    encoder_output = Dropout(0.2, name='encoder_output')(output_lstm)

    # encoder_lstm4 = LSTM(units=50, activation='tanh', return_sequences=True, name='encoder_lstm_4_restore')(encoder_dropout3)
    # encoder_dropout4 = Dropout(0.2, name='encoder_dropout_4_restore')(encoder_lstm4)

    # Attention Layer
    # attention = AttentionLayer(name='attention_layer')(encoder_dropout4)
     # Attention Mechanism
    attention = Dense(1, activation='tanh')(encoder_output)
    attention = Flatten()(attention)
    attention_weights = Activation('softmax')(attention)
    context = Multiply()([encoder_output, Permute([2, 1])(RepeatVector(6)(attention_weights))])

    # Decoder
    decoder_lstm1 = LSTM(units=75, activation='tanh', return_sequences=True, name='decoder_lstm_1_restore')(context)
    decoder_dropout1 = Dropout(0.2, name='decoder_dropout_1_restore')(decoder_lstm1)

    decoder_lstm2 = LSTM(units=25, activation='tanh', return_sequences=True, name='decoder_lstm_2_restore')(decoder_dropout1)
    decoder_dropout2 = Dropout(0.2, name='decoder_dropout_2_restore')(decoder_lstm2)

    decoder_lstm3 = LSTM(units=10, activation='tanh', return_sequences=True, name='decoder_lstm_3_restore')(decoder_dropout2)
    decoder_dropout3 = Dropout(0.2, name='decoder_dropout_3_restore')(decoder_lstm3)

    time_distributed_output = TimeDistributed(Dense(1), name='time_distributed_output')(decoder_dropout3)

    final_output = time_distributed_output[:, -6:, :]

    # Create the model
    model_lstm = Model(inputs=input_layer, outputs=final_output)

    # Compile the model
    optimizer = Adam(learning_rate=0.001)
    model_lstm.compile(optimizer=optimizer, loss="mse")

    return model_lstm




In [1]:
from tensorflow.keras.layers import Input, LSTM, Dropout, Dense, Permute, Multiply, RepeatVector, Lambda, TimeDistributed, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

def attention_mechanism(encoder_out_seq, decoder_out_seq,encoder_hidden_units):
    # encoder_out_seq: (batch_size, time_steps, encoder_hidden_units)
    # decoder_out_seq: (batch_size, decoder_hidden_units)
    
    # Calculating attention scores
    score = Dense(encoder_hidden_units)(decoder_out_seq)
    score = Lambda(lambda x: K.batch_dot(*x, axes=[2, 2]))([score, encoder_out_seq])

    # Softmax to get attention weights
    attention_weights = Activation('softmax')(score)

    # Calculate context vector as weighted sum of encoder outputs
    context_vector = Lambda(lambda x: K.batch_dot(*x, axes=[1, 1]))([attention_weights, encoder_out_seq])
    
    return context_vector

def create_model_with_custom_attention(input_shape, output_steps, latent_dim=6):
    # Encoder
    input_layer = Input(shape=(None, input_shape))

    encoder_lstm1 = LSTM(units=50, activation='tanh', return_sequences=True, name='encoder_lstm_1_restore')(input_layer)
    encoder_dropout1 = Dropout(0.2, name='encoder_dropout_1_restore')(encoder_lstm1)

    encoder_lstm2 = LSTM(units=25, activation='tanh', return_sequences=True, name='encoder_lstm_2_restore')(encoder_dropout1)
    encoder_dropout2 = Dropout(0.2, name='encoder_dropout_2_restore')(encoder_lstm2)

    encoder_lstm3 = LSTM(units=10, activation='tanh', return_sequences=True, name='encoder_lstm_3_restore')(encoder_dropout2)
    encoder_dropout3 = Dropout(0.2, name='encoder_dropout_3_restore')(encoder_lstm3)

    output_lstm = LSTM(units=6, activation='tanh', return_sequences=True, name='outputLSTM')(encoder_dropout3)

    encoder_output = Dropout(0.2, name='encoder_output')(output_lstm)

    # Prepare initial state for decoder (optional, depending on your design)
    decoder_initial_state = Dense(latent_dim)(encoder_output[:,-1,:])

    # Decoder
    all_outputs = []
    decoder_input = RepeatVector(1)(decoder_initial_state)  # initial input for decoder
    for _ in range(output_steps):
        # Attention mechanism
        context_vector = attention_mechanism(encoder_output, decoder_input,6)
        
        # Combine context vector with previous decoder output
        decoder_lstm_input = Concatenate(axis=-1)([context_vector, decoder_input])
        
        # Decoder LSTM followed by dropout
        decoder_output = LSTM(latent_dim, return_sequences=True, return_state=True)(decoder_lstm_input)
        decoder_output, _ = Dropout(0.2)(decoder_output)
        
        # TimeDistributed Dense layer to make the prediction
        decoder_output = TimeDistributed(Dense(1))(decoder_output)
        all_outputs.append(decoder_output)

        # Update decoder_input with the output for next time step
        decoder_input = decoder_output

    # Concatenate all predicted outputs
    final_output = Lambda(lambda x: K.concatenate(x, axis=1))(all_outputs)

    # Create and compile the model
    model = Model(inputs=input_layer, outputs=final_output)
    model.compile(optimizer='adam', loss='mse')

    return model

2024-01-10 23:07:37.982469: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-10 23:07:39.452525: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
def create_model(input_shape, latent_dim=6):
    # Input layer
    input_layer = Input(shape=(None, input_shape))

    # masking_layer = Masking(mask_value=0.0, name='masking_layer')(input_layer)

    # Encoder

    encoder_lstm2 = LSTM(units=50, activation='tanh', return_sequences=True, name='encoder_lstm_2_restore')(input_layer)
    encoder_dropout2 = Dropout(0.2, name='encoder_dropout_2_restore')(encoder_lstm2)

    encoder_lstm3 = LSTM(units=100, activation='tanh', return_sequences=False, name='encoder_lstm_3_restore')(encoder_dropout2)
    encoder_dropout3 = Dropout(0.2, name='encoder_dropout_3_restore')(encoder_lstm3)

    # encoder_lstm4 = LSTM(units=50, activation='tanh', return_sequences=False, name='encoder_lstm_4_restore')(encoder_dropout3)
    # encoder_dropout4 = Dropout(0.2, name='encoder_dropout_4_restore')(encoder_lstm4)

    # Repeat Vector
    repeat_vector = RepeatVector(latent_dim, name='repeat_vector')(encoder_dropout3)

    # Decoder
    decoder_lstm1 = LSTM(units=50, activation='tanh', return_sequences=True, name='decoder_lstm_1_restore')(repeat_vector)
    decoder_dropout1 = Dropout(0.2, name='decoder_dropout_1_restore')(decoder_lstm1)

    decoder_lstm2 = LSTM(units=25, activation='tanh', return_sequences=True, name='decoder_lstm_2_restore')(decoder_dropout1)
    decoder_dropout2 = Dropout(0.2, name='decoder_dropout_2_restore')(decoder_lstm2)

    time_distributed_output = TimeDistributed(Dense(1), name='time_distributed_output')(decoder_dropout2)


    # Create the model
    model_lstm = Model(inputs=input_layer, outputs=time_distributed_output)

    # Compile the model
    optimizer = Adam(learning_rate=0.001)
    model_lstm.compile(optimizer=optimizer, loss="mse")

    return model_lstm


In [ ]:
X_train = test_cluster.X_train_filtered
y_train = test_cluster.y_train
X_test = test_cluster.X_test_filtered
y_test = test_cluster.y_test

start_token = 0

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)



In [ ]:
regular_model = create_model(X_train.shape[2], latent_dim=6)
attention_model = create_model_with_attention2(X_train.shape[2], latent_dim=6)

In [ ]:
import pandas as pd 
import numpy as np
def eval_model(X_test, y_test, model,cluster):

    predicted_y = model.predict(X_test)
    predicted_y = np.squeeze(predicted_y, axis=-1)

    num_days = predicted_y.shape[1]  # Assuming this is the number of days
    results = pd.DataFrame(predicted_y, columns=[f'predicted_{i+1}' for i in range(num_days)])

    for i in range(num_days):
        results[f'real_{i+1}'] = y_test[:, i]

    # Generate output string with accuracies
    output_string = f"Cluster Number: {cluster.label}\n"
    for i in range(num_days):
        same_day = ((results[f'predicted_{i+1}'] > 0) & (results[f'real_{i+1}'] > 0)) | \
                ((results[f'predicted_{i+1}'] < 0) & (results[f'real_{i+1}'] < 0))
        accuracy = round(same_day.mean() * 100,2)

        output_string += (
            f"Accuracy{i+1}D {accuracy}% "
            f"PredictedRet: {results[f'predicted_{i+1}'].mean()} "
            f"ActRet: {results[f'real_{i+1}'].mean()}\n"
        )
    
    output_string += f"Train set length: {len(X_train)} Test set length: {len(y_test)}\n"

    return output_string, results

In [ ]:
from tensorflow.keras.backend import clear_session
clear_session()

In [ ]:
optimizer = Adam(learning_rate=0.0001)
regular_model.compile(optimizer=optimizer, loss='mse')

# Set up early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=35, restore_best_weights=True)

# Fine-tune the model using your smaller dataset
# Assume you have 'small_train_data', 'small_train_labels', 'small_val_data', and 'small_val_labels'
history = regular_model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=250,
                    batch_size=16,
                    callbacks=[early_stopping])

# Evaluate the model performance
val_loss = regular_model.evaluate(X_test, y_test)
regularAccuracy,regularResults = eval_model(X_test, y_test, regular_model,test_cluster)
clear_session()
del regular_model
print(f'Validation loss: {val_loss}')

In [ ]:
optimizer = Adam(learning_rate=0.0001)
attention_model.compile(optimizer=optimizer, loss='mse')

# Set up early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=35, restore_best_weights=True)

# Fine-tune the model using your smaller dataset
# Assume you have 'small_train_data', 'small_train_labels', 'small_val_data', and 'small_val_labels'
history = attention_model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=250,
                    batch_size=16,
                    callbacks=[early_stopping])

# Evaluate the model performance
val_loss = attention_model.evaluate(X_test, y_test)
print(f'Validation loss: {val_loss}')
attentionAccuracy,attentionResults = eval_model(X_test, y_test, attention_model,test_cluster)
clear_session()
del attention_model

In [ ]:
import plotly.graph_objects as go
def visualize_future_distribution(results):
    '''
    Create stacked box and whisker plots for the predicted and real values
    '''

    fig = go.Figure()
    print(results.shape)

    for i in range(6):

        fig.add_trace(go.Box(y=results[f'predicted_{i+1}'], name=f'Predicted {i}')) 
        fig.add_trace(go.Box(y=results[f'real_{i+1}'], name=f'Real {i}'))

    fig.update_layout(
        title='Future Performance of Cluster',
        xaxis_title='Steps in future',
        yaxis_title='Cumulative Percent Change'
    ) 

    return fig

In [ ]:
from plotly.subplots import make_subplots
bench_fig = visualize_future_distribution(regularResults)
tuned_fig = visualize_future_distribution(attentionResults)
fig = make_subplots(rows=1, cols=2)

for trace in bench_fig.data:
    fig.add_trace(trace, row=1, col=1)

for trace in tuned_fig.data:
    fig.add_trace(trace, row=1, col=2)

fig.show()

In [ ]:
print(regularAccuracy)
print(attentionAccuracy)